<a href="https://colab.research.google.com/github/ludoveltz/test_github_fev25/blob/main/Excs_1_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
# Exercise 1 & 2 : Installation et import des bibliothèques
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import pickle

# Exercise 3 : Création et entraînement du modèle
# Génération des données d'exemple
np.random.seed(42)
n_samples = 1000

data = pd.DataFrame({
    'store_nbr': np.random.randint(1, 50, n_samples),
    'family': np.random.choice(['LUXURY', 'PREMIUM_BEAUTY', 'DESIGNER_WEAR', 'ACCESSORIES'], n_samples),
    'store_type': np.random.choice(['FLAGSHIP', 'BOUTIQUE', 'CORNER'], n_samples),
    'transactions': np.random.randint(100, 1000, n_samples),
    'onpromotion': np.random.randint(0, 10, n_samples),
    'sales': np.random.uniform(5000, 50000, n_samples)
})

# Définition des colonnes
categorical_columns = ['family', 'store_type']
numerical_columns = ['store_nbr', 'transactions', 'onpromotion']

# Split des données
X = data.drop('sales', axis=1)
y = data['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Préprocesseurs
num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='constant', fill_value='UNKNOWN')
encoders = {col: LabelEncoder() for col in categorical_columns}
scaler = StandardScaler()

# Prétraitement
X_train_num = num_imputer.fit_transform(X_train[numerical_columns])
X_train_num_scaled = scaler.fit_transform(X_train_num)

X_train_cat = cat_imputer.fit_transform(X_train[categorical_columns])
X_train_cat_encoded = np.zeros((X_train_cat.shape[0], len(categorical_columns)))
for i, col in enumerate(categorical_columns):
    X_train_cat_encoded[:, i] = encoders[col].fit_transform(X_train_cat[:, i])

X_train_final = np.hstack([X_train_num_scaled, X_train_cat_encoded])

# Entraînement du modèle
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_final, y_train)

# Sauvegarde des composants
components = {
    'num_imputer': num_imputer,
    'cat_imputer': cat_imputer,
    'encoders': encoders,
    'scaler': scaler,
    'model': model,
    'categorical_columns': categorical_columns,
    'numerical_columns': numerical_columns
}

with open('rf_model.pkl', 'wb') as f:
    pickle.dump(components, f)

# Exercise 4 : Interface Streamlit
st.set_page_config(page_title="Prédiction des Ventes", layout="wide")

st.title("🎯 Prédiction des Ventes")
st.caption("Analysez et prédisez les ventes")

# Sidebar avec descriptions
st.sidebar.header("Guide des Paramètres")
st.sidebar.markdown("""
**Types de Boutiques**:
- **FLAGSHIP**: Boutiques principales
- **BOUTIQUE**: Points de vente standards
- **CORNER**: Espaces dans les grands magasins

**Familles de Produits**:
- **LUXURY**: Produits haute-gamme
- **PREMIUM_BEAUTY**: Cosmétiques luxe
- **DESIGNER_WEAR**: Prêt-à-porter créateur
- **ACCESSORIES**: Maroquinerie et accessoires
""")

# Interface utilisateur
col1, col2 = st.columns(2)

with col1:
    store_nbr = st.number_input("Numéro de Boutique", 1, 50)
    store_type = st.selectbox("Type de Point de Vente", ['FLAGSHIP', 'BOUTIQUE', 'CORNER'])
    family = st.selectbox("Catégorie", ['LUXURY', 'PREMIUM_BEAUTY', 'DESIGNER_WEAR', 'ACCESSORIES'])

with col2:
    transactions = st.number_input("Transactions", 100, 1000, value=500)
    onpromotion = st.number_input("Articles en Promotion", 0, 10, value=0)

# Exercise 5 : Prédiction
if st.button("Prédire", use_container_width=True):
    try:
        # Préparation des données
        input_data = {
            'store_nbr': store_nbr,
            'store_type': store_type,
            'family': family,
            'transactions': transactions,
            'onpromotion': onpromotion
        }
        input_df = pd.DataFrame([input_data])

        # Prétraitement
        input_df_num = components['num_imputer'].transform(input_df[numerical_columns])
        input_df_cat = components['cat_imputer'].transform(input_df[categorical_columns])

        # Encodage
        input_df_cat_encoded = np.zeros((input_df_cat.shape[0], len(categorical_columns)))
        for i, col in enumerate(categorical_columns):
            input_df_cat_encoded[:, i] = components['encoders'][col].transform(input_df_cat[:, i])

        # Scaling et prédiction
        input_df_num_scaled = components['scaler'].transform(input_df_num)
        input_final = np.hstack([input_df_num_scaled, input_df_cat_encoded])
        prediction = model.predict(input_final)

        # Affichage des résultats
        st.success(f"Prévision des ventes : {prediction[0]:,.2f} €")

    except Exception as e:
        st.error(f"Erreur : {str(e)}")

# Exercise 6 : Lancement de l'application
if __name__ == "__main__":
    from google.colab.output import eval_js
    import time

    # Lancement Streamlit
    !streamlit run app.py &
    time.sleep(3)

    # URL d'accès
    print("\n✨ Application accessible sur :")
    print(eval_js("google.colab.kernel.proxyPort(8501)"))



2025-04-02 13:40:15.581 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 13:40:15.582 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 13:40:15.583 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 13:40:15.584 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 13:40:15.584 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 13:40:15.585 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 13:40:15.586 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-02 13:40:15.587 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.34.41.147:8501

  Stopping...

✨ Application accessible sur :
https://8501-gpu-t4-s-3g9y65y0f988e-c.europe-west4-0.prod.colab.dev
